# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 arxiv==2.3.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 实践代码

为了能够顺利的演示内置中间件的使用详情，这里我们使用一段简单的智能体代码演示：

In [ ]:
from langchain_community.chat_models import ChatTongyi
import os
llm = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")

from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

from langchain.agents import create_agent
agent = create_agent(model=llm, 
                     tools=tools, 
                     system_prompt="You are a helpful assistant", 
                     checkpointer=memory)


论文编号 1605.08386 的信息如下：

- **发表日期**: 2016-05-26
- **标题**: Heat-bath random walks with Markov bases
- **作者**: Caprice Stanley, Tobias Windisch
- **摘要**: 研究了由有限集的允许移动构成的格点图，这些移动可以是任意长度。我们证明了在固定整数矩阵的纤维上，这些图的直径可以从上方由一个常数来限制。然后研究了这些图上的热浴随机游走的混合行为。我们还给出了移动集的显式条件，使得热浴随机游走（Glauber动力学的一种推广）在固定维度下是一个扩展器。


# 2. 智能体安全

## 2.1 PIIMiddleware：隐私信息检测与脱敏

在很多实际的应用场景里，比如企业、教育或医疗等场景中，AI 模型输入/输出中往往包含个人隐私信息（PII），例如：姓名、手机号、邮箱、身份证号、银行卡号、API Key 等。

这些信息其实我们是不希望输入给模型或者模型输出出来的。因此在 LangChain 提供了一个中间键 PIIMiddleware ，其能够通过正则表达式检测的方式，在模型调用前后（before_model 、after_model）自动检测并处理敏感信息，从而确保系统在生成、存储、展示时都符合法律与合规要求（如 GDPR、个人信息保护法等）。
- before_model：检查用户输入、工具输出是否含 PII
- after_model：检查模型输出是否含 PII

首先我们可以创建多个的 PII 检测机制：

In [4]:
from langchain.agents.middleware import PIIMiddleware

pii_middleware = [
  PIIMiddleware("email", strategy="redact", apply_to_input=True),
  PIIMiddleware("ip", strategy="mask", apply_to_input=True),
  PIIMiddleware("url", strategy="block", apply_to_output=True),
]

然后将中间件载入到 agent 中：

In [5]:
agent = create_agent(
  model=llm,
  tools=tools,
  system_prompt="You are a helpful assistant.",
  middleware=pii_middleware, # 👈 核心部分
  checkpointer=memory)

我们可以先测试是否能够发送邮箱（redact 策略）：

In [6]:
config5 = {"configurable": {"thread_id": "user_5"}}
result1 = agent.invoke({"messages": [{"role": "user", "content": "请发送报告到 myemail123@gmail.com"}]}, config=config5)
print(result1["messages"][-1].content)

我无法发送报告到指定的电子邮件地址。请您提供其他方式或确认邮件地址后，我将尽力协助您。


然后测试一下是否能发生 IP 地址（mask 策略）：

In [7]:
result2 = agent.invoke({"messages": [{"role": "user", "content": "服务器地址是 192.168.0.123"}]}, config=config5)
print(result2["messages"][-1].content)

您提供的服务器地址是 *.*.*.123。如果您需要进一步的帮助，请提供更多的信息或说明您的需求。


最后可以测试一下输出时能否带上链接（block 策略）：

In [8]:
try:
  result3 = agent.invoke({"messages": [{"role": "user", "content": "请返回 https://example.com 的摘要"}]}, config=config5)
  print(result3["messages"][-1].content)
except Exception as e:
  print("🚫 检测到 PII 被拦截：", e)

🚫 检测到 PII 被拦截： Detected 1 instance(s) of url in text content


所以可以看到只要我们正确的设置好对应的内容，系统能够自动的拦截或报错一部分的内容。

除了使用默认的 PIIMiddleware 来进行安全防护，我们还可以自己来制作中间件来防护。

比如在 before_model 部分我们自己来检测一下里面是否有禁用词，假如有就返回由于内部有禁用词无法回复，并且跳转到 end 部分：

In [9]:
from typing import Any
from langchain.agents.middleware import before_agent, AgentState, hook_config
from langgraph.runtime import Runtime

banned_keywords = ["hack", "exploit", "malware"]
@before_agent(can_jump_to=["end"])
def content_filter(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Deterministic guardrail: Block requests containing banned keywords."""
    # Get the first user message
    if not state["messages"]:
        return None
    first_message = state["messages"][0]
    if first_message.type != "human":
        return None
    content = first_message.content.lower()
    # Check for banned keywords
    for keyword in banned_keywords:
        if keyword in content:
            # Block execution before any processing
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "I cannot process requests containing inappropriate content. Please rephrase your request."
                }],
                "jump_to": "end"
            }
    return None

 而对于智能体的输出，其实很多时候使用关键词并不那么奏效，因为大模型表达上比较丰富，意思可能都是一样，但是用的词不一样就很难基于关键词进行审查。
 
那其实还有一个很好的办法去审查这种比较主观的内容，就是我们再使用一个比较强的大模型来进行检查内容，让其判定这部分内容是否存在安全问题：

In [ ]:
from langchain.agents.middleware import after_agent
from langchain_core.messages import AIMessage

safety_model = ChatTongyi(model="qwen-max")

@after_agent(can_jump_to=["end"])
def safety_guardrail(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Model-based guardrail: Use an LLM to evaluate response safety."""
    # Get the final AI response
    if not state["messages"]:
        return None
    last_message = state["messages"][-1]
    if not isinstance(last_message, AIMessage):
        return None
    # Use a model to evaluate safety
    safety_prompt = f"""Evaluate if this response is safe and appropriate.
    Respond with only 'SAFE' or 'UNSAFE'.
    Response: {last_message.content}"""
    result = safety_model.invoke([{"role": "user", "content": safety_prompt}])
    if "UNSAFE" in result.content:
        return {
            "messages": [{
                "role": "assistant",
                "content": "I cannot provide that response. Please rephrase your request."
            }],
            "jump_to": "end"
        }
    return None

## 2.2 安全审查 API

除了手动设置安全审查以外，各大大模型厂家其实都提供了一些安全审查的 API，其能够帮助我们快速了解输出中是否存在问题。

比如 OpenAI 里就有一个接口叫 /v1/moderations，其能够 检查文本是否涉及违法、暴力、成人、仇恨、骚扰、自残等类别：

```python
from openai import OpenAI
client = OpenAI(api_key="...")

response = client.moderations.create(
    model="omni-moderation-latest",
    input="I want to hack my school network."
)
print(response.results[0])
```

## 2.3 HumanInTheLoopMiddleware：人工审核

最后还有一种最高规格的安全审查，那就是让人工来参与审核。

其实在很多场景下，假如直接让大模型自己完成所有工作是很难的。这里最主要的原因就是大模型本质上就是一个黑盒模型，会存在幻觉的问题，很可能会做出一些人类无法理解的操作。

当在一些安全敏感的操作，比如修改数据库、发送邮件、调用金融接口等，直接让其修改的话可能会导致信息混乱的情况发生。因此在大模型调用某些特殊工具以前，先让人类进行审核还是非常有必要的。这也是 HumanInTheLoopMiddleware 这个中间键存在的重要原因。

该中间键其实就是让 agent 在执行工具调用之前先暂停，等待人工“批准 / 编辑 / 拒绝”。

有个点需要注意的是，这个必须要设置记忆 checkpointer，否则中断状态无法持久化。

假如我们希望 arxiv 这个工具不要被拦截（默认情况），那我们可以这样来写：

In [11]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "arxiv": False,
    }
)

假如我们希望能够在调用前加上一个人工审核的话，我们可以这样来写：

In [12]:
human_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "arxiv": {  # 工具名
            "allowed_decisions": ["approve", "edit", "reject"]
        }
    }
)

那假如我们希望拦截的时候告诉操作人员该如何判断的话，也可以加上描述的内容：

In [13]:
human_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "arxiv": {
            "allowed_decisions": ["approve", "edit", "reject"],
            "description": "请确认是否调用 arXiv 工具查询论文。",
        }
    },
)

同样的，我们也可以将其载入到 agent 中：

In [14]:
agent = create_agent(
  model=llm,
  tools=tools,
  system_prompt="You are a helpful assistant",
  middleware=[human_middleware],
  checkpointer=InMemorySaver()
)

In [15]:
config6 = {"configurable": {"thread_id": "user_6"}}
result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386，100字即可"}]}, config=config6)
print(result1)

{'messages': [HumanMessage(content='请使用 arxiv 工具查询论文编号 1605.08386，100字即可', additional_kwargs={}, response_metadata={}, id='9640b58b-ea01-497e-bcee-7d4418541f72'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "1605.08386"}', 'name': 'arxiv'}, 'id': 'call_64166476e1104040bc4d28', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': 'a73dfb72-a77e-4c85-b8c6-4a40eb814daf', 'token_usage': {'input_tokens': 238, 'output_tokens': 28, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 266}}, id='lc_run--019b2fc6-dc2d-7dd2-912d-840e56c5a84d-0', tool_calls=[{'name': 'arxiv', 'args': {'query': '1605.08386'}, 'id': 'call_64166476e1104040bc4d28', 'type': 'tool_call'}])], '__interrupt__': [Interrupt(value={'action_requests': [{'name': 'arxiv', 'args': {'query': '1605.08386'}, 'description': '请确认是否调用 arXiv 工具查询论文。'}], 'review_configs': [{'action_name': 'arxiv', 'allowe

那为了能够了解是否发生中断，我们需要先去做一个基本的判定。具体就是看 "__interrupt__" 在不在输出的结果中。

In [16]:
if "__interrupt__" in result1:
  print("🟠 中断已触发，人工审核中...")

🟠 中断已触发，人工审核中...


然后我们再把对应的决策信息（decisions）用 Command 传入到 agent 中：

In [17]:
from langgraph.types import Command

approve_result = agent.invoke(
  Command( 
    resume={"decisions": [{"type": "approve"}]} 
  ), 
  config=config6 # 要和前面对话的 config 进行匹配
)

print(approve_result["messages"][-1].content)

论文编号 1605.08386 的标题是《Heat-bath random walks with Markov bases》，作者是 Caprice Stanley 和 Tobias Windisch，发表于 2016 年 5 月 26 日。该论文研究了基于格点的图及其边的性质，并探讨了热浴随机游走的混合行为。


In [18]:
config6 = {"configurable": {"thread_id": "user_7"}}
result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386，100字即可"}]}, config=config6)

reject_result = agent.invoke(
  Command(
    resume={
      "decisions": [
        {
          "type": "reject",
          "message": "编号不存在"
        }
      ]
    }
  ),
  config=config6
)
print(reject_result["messages"][-1].content)

论文编号 1605.08386 不存在。请检查编号是否正确。


在修改（edit）场景下，假如我们要调整查找的 arxiv 文章，可以按下面的方式实现：

In [19]:

config6 = {"configurable": {"thread_id": "user_8"}}
result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386，100字即可"}]}, config=config6)

edit_result = agent.invoke(Command(resume={
      "decisions": [
        {
          "type": "edit",
          "edited_action": {
            "name": "arxiv",
            "args": {"query": "1605.08387"}
          }
        }
      ]
    }
  ),
  config=config6 
)
print(edit_result["messages"][-1].content)

论文编号 1605.08386 的标题为《Notes on frequencies and timescales in nonequilibrium Green's functions》，作者为 Takaaki Ishii，发表于2016年6月5日。该论文主要研究了非平衡格林函数在强耦合理论中的行为，特别是通过全息对偶关注准正则模的平衡化过程。


但是对于真实的人类介入场景而言，假如我们不知道具体的工具调用细节，我们是无法决定到底应该是 approve 还是 reject 甚至是 edit。

所以我们需要将信息先打印出来，比如获取：
- 用户开始的提问
- 工具的名称
- 工具的传入参数的名称及对应的值
- 中间件中设置的中断描述信息
- 可选择的操作

这样我们就可以基于这些信息决定采取什么样的策略。

除了打印相关的信息以外，还有很重要的一点是我们需要传入一些必要的信息，包括：
- 传入策略信息（approve, reject 或 edit）
    - 对于 approve 而言不需要传入额外信息
    - 对于 reject 的情况需要传入错误反馈
    - 对于 edit 情况需要传入
        - 工具名称
        - 参数名称和对应的值

在 python 的终端中可以使用 input() 函数实现传入。

In [20]:
# ✅ 检查是否触发中断
if "__interrupt__" in result1:
  print("🟠 中断已触发，人工审核中...")

  # 🗣 提取并显示用户提问内容
  user_msg = next((m.content for m in result1["messages"] if m.type == "human"), "无")
  print(f"\n🧑 用户提问：{user_msg}")

  # 🔍 遍历中断详情，展示工具调用信息
  for interrupt in result1["__interrupt__"]:
    for i, req in enumerate(interrupt.value["action_requests"]):
      print(f"\n🔧 工具名：{req['name']}")
      print(f"📦 参数：{req['args']}")
      print(f"📝 描述：{req.get('description', '无')}")
      print(f"✅ 可选操作：{interrupt.value['review_configs'][i]['allowed_decisions']}")

  # 👤 人工输入决策类型
  decision_type = input("\n请输入您的决定（approve / edit / reject）：").strip().lower()
  if decision_type not in ["approve", "edit", "reject"]:
    print("⚠️ 无效输入，默认设置为 reject。")
    decision_type = "reject"

  # 🧩 构造决策对象
  decision_payload = {"type": decision_type}

  # 若是 edit，询问新的参数
  if decision_type == "edit":
    new_query = input("请输入新的论文编号（query）：")
    decision_payload["edited_action"] = {
      "name": "arxiv",
      "args": {"query": new_query}
    }

  # 若是 reject，提供反馈信息
  elif decision_type == "reject":
    decision_payload["message"] = input("请输入拒绝理由：")

  # 🚀 发送人工决策结果
  result2 = agent.invoke(
    Command(resume={"decisions": [decision_payload]}),
    config=config6
  )

  # 📤 展示最终回复
  print("\n🟢 模型后续回复：")
  print(result2["messages"][-1].content)

🟠 中断已触发，人工审核中...

🧑 用户提问：请使用 arxiv 工具查询论文编号 1605.08386，100字即可

🔧 工具名：arxiv
📦 参数：{'query': '1605.08386'}
📝 描述：请确认是否调用 arXiv 工具查询论文。
✅ 可选操作：['approve', 'edit', 'reject']

🟢 模型后续回复：
论文编号 1605.08386 的标题为《Notes on frequencies and timescales in nonequilibrium Green's functions》，作者为 Takaaki Ishii，发表于2016年6月5日。该论文主要研究了非平衡格林函数在强耦合理论中的行为，特别是通过全息对偶关注准正则模的平衡化过程。
